In [ ]:
import os
import pandas as pd
import numpy as np
data_dir = ''
categories = ['CDs_and_Vinyl', 'Grocery_and_Gourmet_Food', 'Toys_and_Games']

In [ ]:
file_path = os.path.join(data_dir, categories[0], 'train', 'product_training.json')
file_path

In [ ]:
c0_product_train = pd.read_json(file_path)
c0_product_train

In [ ]:
file_path = os.path.join(data_dir, categories[0], 'train', 'review_training.json')
c0_review_train = pd.read_json(file_path)
c0_review_train

In [ ]:
c0_review_train.shape

In [ ]:
c0_train = c0_review_train.merge(c0_product_train, how='left', on='asin')

In [15]:
# some reviewText are missing, convert them to empty string

c0_train['reviewText'] = c0_train['reviewText'].fillna('')
c0_train['summary'] = c0_train['summary'].fillna('')
c0_train

,asin,reviewerID,unixReviewTime,vote,verified,reviewTime,style,reviewerName,reviewText,summary,image,awesomeness
0,6673F1740E03573BCD64238FE711FC69,9C856D4A18E1355783B3B28B7ECC1848,1451520000,9,True,"12 31, 2015",{'Format:': ' MP3 Music'},8D88BB79AAC50277AEE82FCFD77F6744,Finding the Beatles all over again - and bette...,I sit listening - with my jaw to the floor - H...,None,1
1,690819436E20BB31657AF6B58B984DD4,6E9ABBD26A27C2B2851D1EC34A01CBDC,1113523200,9,False,"04 15, 2005",{'Format:': ' Audio CD'},8DC0611245A871AC51BBEEBB85F33A58,These guys can sing! Such classic tunes...poi...,Under Appreciated....,None,1
2,A680D4753F0CEA2252C168A6ACB2B623,B637C3C93E61094474710F456928BE9F,1126137600,None,False,"09 8, 2005",None,2259386624CFA0EC53A75A50A9BB57A5,"Snoop Doggy Dogg made a classic album, DoggySt...",DoggyStyle,None,1
3,F4A966F1FA340B16651D676BC246D227,AA7918E9410D650A076221C7B2934A09,954979200,None,False,"04 6, 2000",{'Format:': ' Audio CD'},7A65A155C993535BC99CBCB39E7161B5,Stevie Nicks Has had Her Moments. I Like Some ...,Pretty Good but a Bit Dated,None,1
4,EF59DAF0C00319A48D4784266FD157EE,2293C9B7950A3356B95828419A677720,1477958400,None,True,"11 1, 2016",{'Format:': ' Audio CD'},C69A09446009C500B1364B7DB5510497,Great cd.,Five Stars,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...
770781,BD91503308A437374C3254EDC8BC24CB,936ED23AF4D23943786BBD44D0F1114B,1136246400,12,True,"01 3, 2006",{'Format:': ' Audio CD'},AAB61000438939C8E6165CFCCF02A488,This was the first Simple Minds album that I b...,Their Most Fully Realized Artistic Studio Stat...,None,1
770782,9BC50277D18FAB423AD33C8CE4CC000D,EF922377A87E9D01F50065F2DA1722A8,1290556800,None,False,"11 24, 2010",None,0441BC4F6B7BD180769FDCDD8E603560,I have owned a CD copy of this show for at lea...,Forgettable R&H,None,0
770783,4062627CA1586E517520483964299349,E1F0B0EBC6A36F33301E4FD0B3D62D52,1311120000,3,False,"07 20, 2011",{'Format:': ' Audio CD'},8AB3EEBF23F3583A4396A57DB291D548,carnival of souls to me is bad i gave my cd aw...,Darren d.,None,0
770784,0AE44A6A9176E6A52507B6ABDDA80B00,DDDC81E6B8C3F8C91867F9AECB385135,1111968000,2,False,"03 28, 2005",{'Format:': ' Audio CD'},51B2E62E0A5864316BDB33FD4A729B37,This is an awesome slayer album. I love the th...,awesome slayer cd,None,1


In [ ]:
import pandas as pd
import logging
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm import tqdm

# load the data
data = c0_train

# group the data by asin
grouped_data = data.groupby("asin")

# initialize the sentiment analyzer
sia = SentimentIntensityAnalyzer()

# compute the sentiment scores for each product
product_sentiments = []
for asin, reviews in tqdm(grouped_data):
    try:
        sentiment_scores = []
        for review in reviews["reviewText"]:
            score = sia.polarity_scores(review)["compound"]
            sentiment_scores.append(score)
            logging.info(f'Successfully computed sentiment score for review: {review[:50]}')
        avg_sentiment = sum(sentiment_scores) / len(sentiment_scores) if len(sentiment_scores) > 0 else 0
        product_sentiments.append({"asin": asin, "sentiment": avg_sentiment})
    except Exception as e:
        logging.error(f'Error computing sentiment score for asin {asin}. Error message: {str(e)}')

# create a new dataframe for the sentiment scores
sentiment_df = pd.DataFrame(product_sentiments)

# save the dataframe to a CSV file
sentiment_df.to_csv("product_sentiment.csv", index=False)

# log the successful completion of the program
logging.info("Sentiment analysis completed successfully.")

sentiment_df


In [ ]:
# merge the sentiment_df with c0_train

# read product_sentiment.csv as a dataframe
sentiment_df = pd.read_csv("product_sentiment.csv")

# merge the sentiment_df with c0_train
merged_df = pd.merge(c0_train, sentiment_df, on="asin", how="left")
merged_df

In [ ]:
from tqdm import tqdm
import logging

# group the data by asin
grouped_data = c0_train.groupby("asin")

# calculate the required metrics for each product
product_metrics = []
for asin, reviews in tqdm(grouped_data):
    try:
        num_reviews = len(reviews)
        num_verified_reviews = len(reviews[reviews["verified"] == True])
        num_images_on_reviews = reviews["image"].apply(lambda x: len(x) if x is not None else 0).sum() 
        mean_helpful_votes = reviews["vote"].apply(lambda x: float(x.replace(",", "")) if x is not None else 0).mean()
        mean_review_length = reviews["reviewText"].apply(lambda x: len(x.split())).mean()
        percentage_verified_purchases = num_verified_reviews / num_reviews * 100
        
        product_metrics.append({
            "asin": str(asin), 
            "mean_helpful_votes": mean_helpful_votes,
            "mean_review_length": mean_review_length,
            "percentage_verified_purchases": percentage_verified_purchases,
            "num_images_on_reviews": num_images_on_reviews
        })
    except Exception as e:
        logging.error(f'Error computing metrics for asin {asin}. Error message: {str(e)}')

# create a new dataframe for the metrics
metrics_df = pd.DataFrame(product_metrics)

# save the dataframe to a CSV file
metrics_df.to_csv("product_metrics.csv", index=False)

# log the successful completion of the program
logging.info("Metrics calculation completed successfully.")

In [13]:
metrics_df

,asin,mean_helpful_votes,mean_review_length,percentage_verified_purchases,num_images_on_reviews
0,0000B049F5B33CD310EB1AB236E20191,1.500000,88.500000,50.000000,0
1,00018184A9EC4D270219A296B2580303,2.500000,235.222222,16.666667,0
2,000281A9CAC43FF1F335726A390636DA,1.250000,130.500000,75.000000,0
3,00030884DF109F325638A6BFD5B13CFF,2.862069,79.655172,58.620690,0
4,000325BA25966B5FC701D5D2B5DBA4E0,1.750000,91.500000,100.000000,0
...,...,...,...,...,...
73077,FFFDD3C72D23AF858D6E0ED92612370D,0.890625,169.984375,37.500000,0
73078,FFFDDE284A73B29B320381487EC7DE9E,2.000000,167.250000,50.000000,0
73079,FFFEB3EE2372807964F024707D50FB21,0.000000,120.500000,100.000000,0
73080,FFFF4545AB232D81D0F9B208388BB7AA,1.571429,188.571429,57.142857,0


In [16]:
df = pd.merge(sentiment_df, metrics_df, on="asin", how="left")
df = df.merge(c0_product_train, how='left', on='asin')
df

,asin,sentiment,mean_helpful_votes,mean_review_length,percentage_verified_purchases,num_images_on_reviews,awesomeness
0,0000B049F5B33CD310EB1AB236E20191,0.908350,1.500000,88.500000,50.000000,0,1
1,00018184A9EC4D270219A296B2580303,0.736117,2.500000,235.222222,16.666667,0,0
2,000281A9CAC43FF1F335726A390636DA,0.581325,1.250000,130.500000,75.000000,0,0
3,00030884DF109F325638A6BFD5B13CFF,0.628945,2.862069,79.655172,58.620690,0,1
4,000325BA25966B5FC701D5D2B5DBA4E0,0.856500,1.750000,91.500000,100.000000,0,0
...,...,...,...,...,...,...,...
73077,FFFDD3C72D23AF858D6E0ED92612370D,0.698494,0.890625,169.984375,37.500000,0,1
73078,FFFDDE284A73B29B320381487EC7DE9E,0.949700,2.000000,167.250000,50.000000,0,1
73079,FFFEB3EE2372807964F024707D50FB21,0.955100,0.000000,120.500000,100.000000,0,0
73080,FFFF4545AB232D81D0F9B208388BB7AA,0.510000,1.571429,188.571429,57.142857,0,1


In [20]:
from sklearn.model_selection import train_test_split

# split the data into features and target
# experimented with just having sentiment as feature vector
X = df.drop(columns=["asin", "awesomeness", "mean_helpful_votes", "mean_review_length", "percentage_verified_purchases", "num_images_on_reviews"])
y = df["awesomeness"]

# split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
X_train

,sentiment
26599,0.955925
30252,0.563700
11003,0.821200
55888,0.666280
14277,0.753400
...,...
37194,0.340450
6265,0.788600
54886,0.697963
860,0.909000


In [23]:
X_test

,sentiment
47430,0.915320
64537,0.654954
47624,0.743933
72837,0.548988
29973,0.916067
...,...
53549,0.681985
31355,0.566850
27342,0.686471
53397,0.803800


In [24]:
y_train

26599    1
30252    1
11003    1
55888    1
14277    1
        ..
37194    1
6265     1
54886    0
860      0
15795    0
Name: awesomeness, Length: 58465, dtype: int64

In [25]:
y_test

47430    1
64537    1
47624    1
72837    0
29973    1
        ..
53549    1
31355    0
27342    1
53397    0
19927    1
Name: awesomeness, Length: 14617, dtype: int64

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# fit classifiers and make predictions on test set
classifiers = [GaussianNB(), KNeighborsClassifier(), DecisionTreeClassifier(), SVC(), LogisticRegression(), RandomForestClassifier()]
classifier_names = ["Naive Bayes", "K-NN", "Decision Trees", "SVM", "Logistic Regression", "Random Forest"]
for classifier, name in tqdm(zip(classifiers, classifier_names), total=len(classifiers)):
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)

    # compute evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    # print results
    print(f"Results for {name}:")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 score: {f1}")
    print(f"Confusion matrix:\n{cm}")

 17%|█▋        | 1/6 [00:00<00:00,  8.07it/s]

Results for Naive Bayes:
Accuracy: 0.5524389409591571
Precision: 0.5499446815236289
Recall: 0.8914937227773507
F1 score: 0.6802541544477028
Confusion matrix:
[[1116 5695]
 [ 847 6959]]


 33%|███▎      | 2/6 [00:01<00:02,  1.79it/s]

Results for K-NN:
Accuracy: 0.5152220017787508
Precision: 0.5438062788999757
Recall: 0.5725083269280041
F1 score: 0.5577883175237145
Confusion matrix:
[[3062 3749]
 [3337 4469]]


 50%|█████     | 3/6 [00:02<00:01,  1.55it/s]

Results for Decision Trees:
Accuracy: 0.5063282479304919
Precision: 0.5407796516450096
Recall: 0.5011529592621061
F1 score: 0.5202127659574468
Confusion matrix:
[[3489 3322]
 [3894 3912]]


 83%|████████▎ | 5/6 [07:16<01:31, 91.44s/it] 

Results for SVM:
Accuracy: 0.5556543750427584
Precision: 0.5544118867767909
Recall: 0.855623879067384
F1 score: 0.6728454137913665
Confusion matrix:
[[1443 5368]
 [1127 6679]]
Results for Logistic Regression:
Accuracy: 0.5544913456933708
Precision: 0.552278603749192
Recall: 0.8756085062772226
F1 score: 0.677336240214052
Confusion matrix:
[[1270 5541]
 [ 971 6835]]


100%|██████████| 6/6 [07:50<00:00, 78.43s/it]

Results for Random Forest:
Accuracy: 0.5073544502975987
Precision: 0.537675924772699
Recall: 0.5530361260568794
F1 score: 0.5452478686454059
Confusion matrix:
[[3099 3712]
 [3489 4317]]
